# PySpark SCD2 Change Capture Application
This notebook demonstrates how to read snapshotted Parquet files, process each snapshot, and maintain a Slowly Changing Dimension Type 2 (SCD2) Parquet file using PySpark.

In [1]:
# Install Required Libraries
!pip install pyspark pandas numpy

     ---------------------------------------- 0.0/434.1 MB ? eta -:--:--
     --------------------------------------- 3.4/434.1 MB 22.3 MB/s eta 0:00:20
     --------------------------------------- 4.7/434.1 MB 13.0 MB/s eta 0:00:34
      --------------------------------------- 5.8/434.1 MB 9.8 MB/s eta 0:00:44
      --------------------------------------- 6.8/434.1 MB 8.7 MB/s eta 0:00:49
      --------------------------------------- 7.6/434.1 MB 8.0 MB/s eta 0:00:54
      --------------------------------------- 8.1/434.1 MB 6.9 MB/s eta 0:01:02
      --------------------------------------- 8.4/434.1 MB 6.3 MB/s eta 0:01:08
      --------------------------------------- 8.9/434.1 MB 5.7 MB/s eta 0:01:16
      --------------------------------------- 9.7/434.1 MB 5.3 MB/s eta 0:01:21
      -------------------------------------- 10.5/434.1 MB 5.1 MB/s eta 0:01:23
     - ------------------------------------- 11.3/434.1 MB 5.0 MB/s eta 0:01:25
     - ------------------------------------- 12

In [9]:
# Import Libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from pyspark.sql.window import Window
import pandas as pd
import numpy as np
import os, re, datetime, time

In [10]:
# Verify Library Installation
print('pandas version:', pd.__version__)
print('numpy version:', np.__version__)

pandas version: 2.3.1
numpy version: 2.3.2


## SCD2 Change Capture Logic
The following cells implement the logic to read snapshotted Parquet files, process each snapshot, and maintain a SCD2 Parquet file.

In [15]:
# SCD2 Change Capture Implementation
spark = SparkSession.builder.appName("SCD2ChangeCapture").getOrCreate()

import glob
import re

snapshot_folder = r"C:\codes\practice\spark-practice\datastore\applicant"
scd2_path = r"change-capture\scd2_output.parquet"

# Find all Parquet files and infer snapshot date from subfolder name
parquet_files = glob.glob(snapshot_folder + "*/**.parquet", recursive=True)

snapshots = []
for file in parquet_files:
    # Extract snapshot date from folder name
    match = re.search(r"applicant\\([0-9]{4}[0-9]{2}[0-9]{2})", file)
    if match:
        snapshot_date = match.group(1)
        df = spark.read.parquet(file)
        df = df.withColumn("snapshot_date", F.lit(snapshot_date))
        snapshots.append(df)



In [17]:
df = spark.read.parquet(r"C:\codes\practice\spark-practice\datastore\applicant")

Py4JJavaError: An error occurred while calling o68.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:817)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1415)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1620)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:739)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2078)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2122)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:961)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:218)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:132)
	at scala.collection.immutable.List.map(Li

In [12]:
df.show(100,False)

+-----------+--------+------+---------------------------------------------------------+------+----+-----+-----+----------------+--------+-----------+--------+
|PassengerId|Survived|Pclass|Name                                                     |Sex   |Age |SibSp|Parch|Ticket          |Fare    |Cabin      |Embarked|
+-----------+--------+------+---------------------------------------------------------+------+----+-----+-----+----------------+--------+-----------+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                                  |male  |22.0|1    |0    |A/5 21171       |7.25    |NULL       |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)      |female|38.0|1    |0    |PC 17599        |71.2833 |C85        |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                                   |female|26.0|0    |0    |STON/O2. 3101282|7.925   |NULL       |S       |
|4          |1       |1     |Futrelle, Mrs. Ja

In [ ]:
# Union all snapshots into one DataFrame
if snapshots:
    all_snapshots_df = snapshots[0]
    for df in snapshots[1:]:
        all_snapshots_df = all_snapshots_df.unionByName(df)
else:
    all_snapshots_df = spark.createDataFrame([], schema=None)  # Empty DataFrame

# Window spec for SCD2
window_spec = Window.partitionBy("id").orderBy("snapshot_date")

# Detect changes by comparing with previous snapshot
changed_df = all_snapshots_df.withColumn("prev_field1", F.lag("field1").over(window_spec)) \
    .withColumn("prev_field2", F.lag("field2").over(window_spec))

scd2_df = changed_df.withColumn(
    "is_current",
    F.when((F.col("field1") != F.col("prev_field1")) | (F.col("field2") != F.col("prev_field2")) | F.col("prev_field1").isNull(), F.lit(True)).otherwise(F.lit(False))
).withColumn("start_date", F.col("snapshot_date"))

scd2_df = scd2_df.withColumn(
    "end_date",
    F.when(F.col("is_current") == False, F.col("snapshot_date")).otherwise(F.lit(None))
)

scd2_df.write.mode("overwrite").parquet(scd2_path)
spark.stop()